# Define S3 client

In [ ]:
import os

s3endpoint = os.environ['s3endpoint']
# s3bucket = os.environ['s3bucket_models']
bucket_models = os.environ['s3bucket_models']
bucket_ready  = os.environ['s3bucket_ready']
mount_path = os.environ['working_dir']

In [2]:
import os
import boto3
from botocore.client import Config

# s3_client = boto3.client('s3',
#         # endpoint_url='https://minio-api-test02.apps.rhods-internal.61tk.p1.openshiftapps.com',
#         endpoint_url='https://minio-api-test03.apps.rhods-internal.61tk.p1.openshiftapps.com',
#         aws_access_key_id='minio',
#         aws_secret_access_key='minio123',
#         config=Config(signature_version='s3v4'),
#         region_name='us-east-1')

# s3endpoint = "https://minio-api-central.apps.cluster-2lngl.sandbox1314.opentlc.com"
# s3bucket = "production"


s3_client = boto3.client('s3',
        endpoint_url=s3endpoint,
        aws_access_key_id='minio',
        aws_secret_access_key='minio123',
        config=Config(signature_version='s3v4'),
        region_name='us-east-1')

      

# Calculate next version

In [26]:
objects = s3_client.list_objects_v2(Bucket=bucket_models)

version = 0

if "Contents" in objects:
    for obj in objects['Contents']:
        print(obj['Key'])
        s3version = int(obj['Key'].split('/')[2])
        if s3version > version:
            version = s3version
else:
    print("there are NO contents")
    
version += 1
version

models/tea_model_b64/1/fingerprint.pb
models/tea_model_b64/1/saved_model.pb
models/tea_model_b64/1/variables/variables.data-00000-of-00001
models/tea_model_b64/1/variables/variables.index
models/tea_model_b64/5/fingerprint.pb
models/tea_model_b64/5/saved_model.pb
models/tea_model_b64/5/variables/variables.data-00000-of-00001
models/tea_model_b64/5/variables/variables.index


6

# Push model to Repository

In [20]:
# path = "models/tea_model_b64"
path = mount_path+"models/tea_model_b64"

for subdir, dirs, files in os.walk(path):
    for file in files:
        full_path = os.path.join(subdir, file)
        s3_path = os.path.join(subdir.removeprefix(mount_path), file)
        s3_path = s3_path.replace("/1/","/"+str(version)+"/")
        print(s3_path)
        s3_client.upload_file(full_path, bucket_models, s3_path)
        

# Push model for (edge) collection

In [20]:
# path = "models/tea_model_b64"
path = mount_path+"models/tea_model_b64"

for subdir, dirs, files in os.walk(path, topdown=False):
    for file in files:
        full_path = os.path.join(subdir, file)
        s3_path = os.path.join(subdir.removeprefix(mount_path), file)
        s3_path = s3_path.replace("/1/","/"+str(version)+"/")
        print(s3_path)
        s3_client.upload_file(full_path, bucket_ready,  s3_path)
        